In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from LSTM import *
from CNN import *
from torchinfo import summary

In [4]:
model=classifier_LSTM(relup=True,
                    input_size = 96,
                    lstm_layers = 1,
                    lstm_size = 128,
                    output1_size = 128,
                    output2_size = 100,
                    GPUindex = 1)


In [7]:
cnn = classifier_CNN(in_channel = 96,
                    num_points = 512,
                    output_size = 100)

In [5]:
print(summary(model,input_size=(1, 512, 96)))

torch.Size([1, 512, 96])
Layer (type:depth-idx)                   Output Shape              Param #
classifier_LSTM                          [1, 100]                  --
├─LSTM: 1-1                              [1, 512, 128]             115,712
├─Linear: 1-2                            [1, 128]                  16,512
├─ReLU: 1-3                              [1, 128]                  --
├─Linear: 1-4                            [1, 100]                  12,900
Total params: 145,124
Trainable params: 145,124
Non-trainable params: 0
Total mult-adds (M): 59.27
Input size (MB): 0.20
Forward/backward pass size (MB): 0.53
Params size (MB): 0.58
Estimated Total Size (MB): 1.30


In [9]:
print(summary(cnn,input_size=(1, 512, 96)))

Layer (type:depth-idx)                   Output Shape              Param #
classifier_CNN                           [1, 100]                  --
├─Conv1d: 1-1                            [96, 8, 481]              264
├─ELU: 1-2                               [96, 8, 481]              --
├─Dropout: 1-3                           [1, 481, 768]             --
├─Linear: 1-4                            [1, 481, 40]              30,760
├─Dropout: 1-5                           [1, 481, 40]              --
├─AvgPool1d: 1-6                         [1, 40, 6]                --
├─Linear: 1-7                            [1, 100]                  24,100
Total params: 55,124
Trainable params: 55,124
Non-trainable params: 0
Total mult-adds (M): 12.25
Input size (MB): 0.20
Forward/backward pass size (MB): 3.11
Params size (MB): 0.22
Estimated Total Size (MB): 3.53
